In [2]:
!pip install python-socketio

In [4]:
!pip install eventlet

  Using cached monotonic-1.5-py2.py3-none-any.whl (5.3 kB)
  Using cached dnspython-1.16.0-py2.py3-none-any.whl (188 kB)


In [6]:
!pip install flask

In [8]:
!pip install python-dotenv

In [10]:
import socketio
import eventlet
import numpy as np
from flask import Flask
from keras.models import load_model
import base64
from io import BytesIO
from PIL import Image
import cv2

Using TensorFlow backend.
C:\Users\rasumit\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\rasumit\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\rasumit\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\rasumit\Anaconda3\lib\site-packages\tensorflow\python\framework\d

In [11]:
sio = socketio.Server()

In [12]:
app = Flask(__name__) #'__main__'
speed_limit = 10
def img_preprocess(img):
    img = img[60:135,:,:]
    img = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
    img = cv2.GaussianBlur(img,  (3, 3), 0)
    img = cv2.resize(img, (200, 66))
    img = img/255
    return img

In [13]:
@sio.on('telemetry')
def telemetry(sid, data):
    speed = float(data['speed'])
    image = Image.open(BytesIO(base64.b64decode(data['image'])))
    image = np.asarray(image)
    image = img_preprocess(image)
    image = np.array([image])
    steering_angle = float(model.predict(image))
    throttle = 1.0 - speed/speed_limit
    print('{} {} {}'.format(steering_angle, throttle, speed))
    send_control(steering_angle, throttle)

In [14]:
@sio.on('connect')
def connect(sid, environ):
    print('Connected')
    send_control(0, 0)
 
def send_control(steering_angle, throttle):
    sio.emit('steer', data = {
        'steering_angle': steering_angle.__str__(),
        'throttle': throttle.__str__()
    })
 

In [ ]:
if __name__ == '__main__':
    model = load_model('model.h5')
    app = socketio.Middleware(sio, app)
    eventlet.wsgi.server(eventlet.listen(('', 4567)), app)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


(60528) wsgi starting up on http://0.0.0.0:4567
(60528) accepted ('127.0.0.1', 65113)


Connected
-0.22694943845272064 1.0 0.0
-0.22694943845272064 1.0 0.0
-0.22694329917430878 1.0 0.0
-0.22697605192661285 0.98083 0.1917
-0.22192658483982086 0.9352199999999999 0.6478
-0.22317053377628326 0.8855 1.145
-0.22222290933132172 0.84272 1.5728
-0.21988223493099213 0.79792 2.0208
-0.2182585746049881 0.73597 2.6403
-0.21404390037059784 0.71675 2.8325
-0.20712579786777496 0.67665 3.2335
-0.20825476944446564 0.6573 3.427
-0.20030094683170319 0.62273 3.7727
-0.19806446135044098 0.58962 4.1038
-0.19668684899806976 0.5584399999999999 4.4156
-0.1852331906557083 0.5134299999999999 4.8657
-0.17774994671344757 0.4851 5.149
-0.17722128331661224 0.45848 5.4152
-0.17560504376888275 0.42195000000000005 5.7805
-0.17556481063365936 0.4112 5.888
-0.16514413058757782 0.3795200000000001 6.2048
-0.15782763063907623 0.35967000000000005 6.4033
-0.1472821682691574 0.34162000000000003 6.5838
-0.15002699196338654 0.31628 6.8372
-0.14040382206439972 0.30064 6.9936
-0.1357942670583725 0.27895000000000003 7.

-0.055366646498441696 0.07828999999999997 9.2171
-0.048972856253385544 0.07828999999999997 9.2171
-0.048972856253385544 0.07828999999999997 9.2171
-0.035104673355817795 0.07828999999999997 9.2171
-0.035104673355817795 0.07827000000000006 9.2173
-0.025780927389860153 0.07827000000000006 9.2173
-0.025780927389860153 0.07827000000000006 9.2173
-0.008078795857727528 0.07826 9.2174
-0.008078795857727528 0.07825000000000004 9.2175
-0.008078795857727528 0.07825000000000004 9.2175
-0.006135773845016956 0.07824000000000009 9.2176
-0.006135773845016956 0.07824000000000009 9.2176
-0.0033718766644597054 0.07824000000000009 9.2176
-0.00044603366404771805 0.07822999999999991 9.2177
0.013578074984252453 0.07822999999999991 9.2177
0.014384406618773937 0.07821999999999996 9.2178
0.031036842614412308 0.07821999999999996 9.2178
0.020830918103456497 0.07821999999999996 9.2178
0.02763580158352852 0.07821999999999996 9.2178
0.02763580158352852 0.07824000000000009 9.2176
0.017754394561052322 0.07826 9.2174
0

-0.006562006659805775 0.07821 9.2179
-0.00919381994754076 0.07820000000000005 9.218
-0.008149725385010242 0.07820000000000005 9.218
-0.018370788544416428 0.07820000000000005 9.218
-0.01609332486987114 0.07820000000000005 9.218
-0.012392145581543446 0.07818999999999998 9.2181
-0.013228011317551136 0.07818999999999998 9.2181
-0.006755751557648182 0.07820000000000005 9.218
-0.0022081853821873665 0.07818999999999998 9.2181
0.0006756363436579704 0.07818999999999998 9.2181
-0.002621960826218128 0.07818999999999998 9.2181
-0.0002739550545811653 0.07818999999999998 9.2181
0.00264014583081007 0.07818999999999998 9.2181
0.012948918156325817 0.07818999999999998 9.2181
0.010258483700454235 0.07818999999999998 9.2181
0.013254034332931042 0.07818999999999998 9.2181
0.01171835046261549 0.07818999999999998 9.2181
0.0051739392802119255 0.07820000000000005 9.218
0.018789280205965042 0.07820000000000005 9.218
0.0145809231325984 0.07820000000000005 9.218
0.0186186321079731 0.07820000000000005 9.218
0.0189

...

-0.048399727791547775 0.07844999999999991 9.2155
-0.045468222349882126 0.07847000000000004 9.2153
-0.05035005137324333 0.0784800000000001 9.2152
-0.048786114901304245 0.07850000000000001 9.215
-0.04458741471171379 0.07851999999999992 9.2148
-0.04285744950175285 0.07851999999999992 9.2148
-0.05179227516055107 0.07852999999999999 9.2147
-0.047182124108076096 0.07855000000000012 9.2145
-0.047182124108076096 0.07851999999999992 9.2148
-0.05633265897631645 0.07850000000000001 9.215
-0.04505876824259758 0.07845999999999997 9.2154
-0.038250457495450974 0.07843999999999995 9.2156
-0.038927506655454636 0.07841000000000009 9.2159
-0.03218422457575798 0.07840000000000003 9.216
-0.032590221613645554 0.07836999999999994 9.2163
-0.041822951287031174 0.07835000000000003 9.2165
-0.04275594279170036 0.07833000000000001 9.2167
-0.04396415874361992 0.07830999999999988 9.2169
-0.04963214322924614 0.07828999999999997 9.2171
-0.05560128018260002 0.07828999999999997 9.2171
-0.06177176162600517 0.078280000000

-0.08596913516521454 0.07825000000000004 9.2175
-0.09272141754627228 0.07826 9.2174
-0.08605526387691498 0.07827000000000006 9.2173
-0.09582649171352386 0.07829999999999993 9.217
-0.08370546996593475 0.07832000000000006 9.2168
-0.08789874613285065 0.07834000000000008 9.2166
-0.0849878340959549 0.07834000000000008 9.2166
-0.08824621140956879 0.07835000000000003 9.2165
-0.0763014405965805 0.07833000000000001 9.2167
-0.0718517154455185 0.07833000000000001 9.2167
-0.07465507090091705 0.07832000000000006 9.2168
-0.06756819784641266 0.07829999999999993 9.217
-0.07011254131793976 0.07828999999999997 9.2171
-0.06387500464916229 0.07827000000000006 9.2173
-0.06013793870806694 0.07825000000000004 9.2175
-0.060097139328718185 0.07825000000000004 9.2175
-0.04894314333796501 0.07822999999999991 9.2177
-0.044358592480421066 0.07821 9.2179
-0.04018257185816765 0.07820000000000005 9.218
-0.040158700197935104 0.07818999999999998 9.2181
-0.0409347228705883 0.07818000000000003 9.2182
-0.0409347228705883 

0.0013105450198054314 0.07826 9.2174
-0.002138596959412098 0.07826 9.2174
-0.01279954332858324 0.07826 9.2174
-0.01741991564631462 0.07825000000000004 9.2175
-0.027067136019468307 0.07825000000000004 9.2175
-0.019527297466993332 0.07825000000000004 9.2175
-0.017909687012434006 0.07825000000000004 9.2175
-0.029670637100934982 0.07826 9.2174
-0.028927069157361984 0.07827000000000006 9.2173
-0.033736955374479294 0.07826 9.2174
-0.033736955374479294 0.07827000000000006 9.2173
-0.03908641263842583 0.07827000000000006 9.2173
-0.03908641263842583 0.07827000000000006 9.2173
-0.0370071642100811 0.07827000000000006 9.2173
-0.0370047502219677 0.07827000000000006 9.2173
-0.032121431082487106 0.07827000000000006 9.2173
-0.026837479323148727 0.07827000000000006 9.2173
-0.02348608896136284 0.07826 9.2174
-0.019232582300901413 0.07825000000000004 9.2175
-0.020943503826856613 0.07825000000000004 9.2175
-0.028915774077177048 0.07824000000000009 9.2176
-0.029984038323163986 0.07822999999999991 9.2177
-0.

-0.08700843155384064 0.07867999999999997 9.2132
-0.08307914435863495 0.07864000000000004 9.2136
-0.07988069951534271 0.07860999999999996 9.2139
-0.08572559058666229 0.07857999999999998 9.2142
-0.08888144791126251 0.07853999999999994 9.2146
-0.09154205024242401 0.07849000000000006 9.2151
-0.09798033535480499 0.07844999999999991 9.2155
-0.09211106598377228 0.07844999999999991 9.2155
-0.09954844415187836 0.07843 9.2157
-0.09035356342792511 0.07842000000000005 9.2158
-0.08150093257427216 0.07842000000000005 9.2158
-0.0789913684129715 0.07841000000000009 9.2159
-0.08024357259273529 0.07878000000000007 9.2122
-0.06894494593143463 0.07749000000000006 9.2251
-0.06933148205280304 0.07782999999999995 9.2217
-0.06446252763271332 0.07827000000000006 9.2173
-0.05581377074122429 0.07851999999999992 9.2148
-0.055355917662382126 0.0787000000000001 9.213
-0.05334250256419182 0.07879999999999998 9.212
-0.06486926972866058 0.07889000000000002 9.2111
-0.05676962062716484 0.07895000000000008 9.2105
-0.0638

127.0.0.1 - - [13/Jul/2020 22:06:03] "GET /socket.io/?EIO=4&transport=websocket HTTP/1.1" 200 0 107.828747
